In [1]:
# xMD testing
import pandas as pd
import os
from xMD.xMD import xMD
from xMD.MD_Settings import GROMACS_Settings

settings = GROMACS_Settings()


/home/alexi/Documents/xMD


In [2]:




amber14sb_ff_path = os.path.join(os.getcwd())

# Set the GMXLIB environment variable
os.environ["GMXLIB"] = amber14sb_ff_path



In [3]:
settings = GROMACS_Settings()
settings.suffix = "APO_amber99"
# settings.search = "APO"

print(settings.config)

settings.topology = os.path.join(settings.topology,"BPTI")
print(settings.topology)
# make sure to turn on MPI for HPC 
settings.gmx_mpi_on = False


config
topology/BPTI


In [4]:
# list config dir 

configs = os.listdir(settings.config)

configs = [config if ("_" in config and ".mdp" in config) else None for config in configs]
configs = [config for config in configs if config is not None]

print(sorted(configs))

['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp', '7_prod_10ps_out.mdp', 'md_short.mdp']


In [7]:
# specify ARGS: -P, -R, -N
md = xMD(settings, 'BPTI_genvel1', "5PTI", 1)

mdps = ['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp']

file_names = ["5PTI_APO_amber99_H_solv_ions.gro", "5PTI_APO_amber99_H.top", "5PTI_APO_amber99_H.itp"]
# md.check_args()
md.create_directory_structure(overwrite=True)
md.run_experiment(config_files=mdps,topology_files=file_names)

save_path = md.save_experiment()

Replicate number:  1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Environment variables set:  GMXLIB /home/alexi/Documents/xMD
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Creating directory:  R_1 Creating directory:  R_2 Creating directory:  R_3 

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx grompp -f config/1_minim.mdp -c data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro -p data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H.top -o data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_0.tpr -r data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro -maxwarn 1 -v

Ignoring obsolete mdp entry 'ns-type'

NOTE 1 [file config/1_minim.mdp]:
  You have set rlist larger than the interaction cut-off, but you also have
  verlet-buffer-tolerance > 0. Will set rlist using
  verlet-buffer-tolerance.


NOTE 2 [file config/1_minim.mdp]:
  Setting tcoupl from 'V-rescale' to 'no'. Temperature coupling does not
  apply to steep.


NOTE 3 [file config/1_minim.mdp]:
  Setting pcoupl from 'Parrinello-Rahman' to 'no'. Pressure coupling does
  not apply to steep.

Generating 1-4 interactions: f

CalledProcessError: Command '['gmx', 'grompp', '-f', 'config/1_minim.mdp', '-c', 'data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro', '-p', 'data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H.top', '-o', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_0.tpr', '-r', 'data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro', '-maxwarn', '1', '-v']' returned non-zero exit status 1.

In [ ]:
def parse_gromacs_energy_log(filepath):
    """
    Parse a GROMACS log file to extract energy information at each step into a DataFrame.
    """
   # read the file
    with open(filepath, 'r') as f:
        log = f.readlines()
    
    # find the start of the energy table

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            # find:            Step           Time
            if len(split)==2 and split[0] == "Step" and split[1] == "Time":
                start = i
                break
            #find end

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            if len(split)>1 and split[0] == "Statistics":
                end = i
                break

        # extract the table
        table = log[start+1:end]
    # extract the headers
    df = pd.DataFrame()

    return df


log_file_path = "data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log"
# Parse the log file again with the updated function
energy_df_updated = parse_gromacs_energy_log(log_file_path)
